In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

DATA_PATH = 'C:/Users/routp/Downloads/skinIMG_split'
IMG_SIZE = (128,128)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,  
    shear_range = 0.2,
    zoom_range = 0.2,   
    horizontal_flip = True,
)
test_val_datagen = ImageDataGenerator(rescale = 1./255)

train_gen = test_val_datagen.flow_from_directory(
    os.path.join(DATA_PATH, 'train'),
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
)
val_gen = test_val_datagen.flow_from_directory(
    os.path.join(DATA_PATH, 'validation'), 
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
)
test_gen = test_val_datagen.flow_from_directory(    
    os.path.join(DATA_PATH, 'test'),
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = False
)
num_classes = train_gen.num_classes
print(f"Number of classes: {num_classes}")

c:\Users\routp\Desktop\health_dignosis\server\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\routp\Desktop\health_dignosis\server\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\routp\Desktop\health_dignosis\server\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_

Found 17728 images belonging to 8 classes.
Found 3796 images belonging to 8 classes.
Found 3807 images belonging to 8 classes.
Number of classes: 8


MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

bsae_model = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
)
for layer in bsae_model.layers:
    layer.trainable = False

x = bsae_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x= Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs = bsae_model.input, outputs = output)


In [4]:
import tensorflow as tf

# Define a learning rate schedule
# This will decrease the learning rate every few epochs
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5, # A very small starting learning rate
    decay_steps=100000,
    decay_rate=0.9
)

# Recompile the model with the learning rate schedule
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Now, fit the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30 # Set a higher number of epochs for fine-tuning
)

c:\Users\routp\Desktop\health_dignosis\server\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
554/554 ━━━━━━━━━━━━━━━━━━━━ 1486s 3s/step - accuracy: 0.5125 - loss: 1.4699 - val_accuracy: 0.5595 - val_loss: 1.2824
Epoch 2/30
554/554 ━━━━━━━━━━━━━━━━━━━━ 1501s 3s/step - accuracy: 0.5534 - loss: 1.3207 - val_accuracy: 0.5806 - val_loss: 1.2159
Epoch 3/30
554/554 ━━━━━━━━━━━━━━━━━━━━ 1463s 3s/step - accuracy: 0.5740 - loss: 1.2677 - val_accuracy: 0.5925 - val_loss: 1.1817
Epoch 4/30
554/554 ━━━━━━━━━━━━━━━━━━━━ 1461s 3s/step - accuracy: 0.5833 - loss: 1.2262 - val_accuracy: 0.5998 - val_loss: 1.1591
Epoch 5/30
554/554 ━━━━━━━━━━━━━━━━━━━━ 1490s 3s/step - accuracy: 0.5912 - loss: 1.2111 - val_accuracy: 0.6014 - val_loss: 1.1426
Epoch 6/30
554/554 ━━━━━━━━━━━━━━━━━━━━ 1450s 3s/step - accuracy: 0.5984 - loss: 1.1876 - val_accuracy: 0.6072 - val_loss: 1.1288
Epoch 7/30
554/554 ━━━━━━━━━━━━━━━━━━━━ 1467s 3s/step - accuracy: 0.6010 - loss: 1.1756 - val_accuracy: 0.6101 - val_loss: 1.1155
Epoch 8/30
554/554 ━━━━━━━━━━━━━━━━━━━━ 1454s 3s/step - accuracy: 0.6056 - loss: 1.1567 - 

In [4]:
# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_gen)

print(f"\nFinal Test Loss: {test_loss:.4f}")
print(f"Final Test Accuracy: {test_acc*100:.2f}%")

ValueError: You must call `compile()` before using the model.


Model saved successfully as 'binary_cancer_model.h5'
